In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

from catboost import CatBoostClassifier

import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('./data/flight/flight_delays_train.csv')
test_df = pd.read_csv('./data/flight/flight_delays_test.csv')

In [3]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [5]:
train_df.shape,test_df.shape

((100000, 9), (100000, 8))

In [6]:
def cleanTime(x):
    if len(str(x))==4 and int(str(x)[:2])>=24:
        return str(int(str(x)[:2])-24)+str(x)[2:]
    elif len(str(x)) > 1:
        return str(x)
    else:
        return "0"+str(x)

In [7]:
def createtimefeatures(df, converstion=True):
    df["DepTime"] = df.DepTime.map(lambda x: cleanTime(x))
    df['hour'] = df.DepTime.map(lambda x: pd.to_datetime(x,format='%H%M').hour)
    df['min'] = df.DepTime.map(lambda x: pd.to_datetime(x,format='%H%M').minute)
    
    if converstion:
        df['sin_hour'] = np.sin(2*np.pi*df['hour']/24)
        df['cos_hour'] = np.cos(2*np.pi*df['hour']/24)

        df['sin_min'] = np.sin(2*np.pi*df['min']/60)
        df['cos_min'] = np.cos(2*np.pi*df['min']/60)
    
        df.drop(["hour", "min" ], axis=1,inplace=True)
        
    df.drop(["DepTime"], axis=1,inplace=True)
    return df
    

In [8]:
train_df = createtimefeatures(train_df)
test_df = createtimefeatures(test_df)

In [48]:
# ss = StandardScaler()
# train_df['Distance'] = ss.fit_transform(train_df.Distance.values.reshape(-1,1))
# test_df['Distance'] = ss.transform(test_df.Distance.values.reshape(-1,1))

In [9]:
train_df['DayMonth'] = train_df.apply(lambda x: x['Month'] +" "+ x['DayofMonth'], axis=1 )
train_df['MonthWeek'] = train_df.apply(lambda x: x['Month'] +" "+ x['DayOfWeek'], axis=1 )

train_df.drop(['Month', 'DayofMonth','DayOfWeek'],axis=1, inplace=True)

In [10]:
test_df['DayMonth'] = test_df.apply(lambda x: x['Month'] +" "+ x['DayofMonth'], axis=1 )
test_df['MonthWeek'] = test_df.apply(lambda x: x['Month'] +" "+ x['DayOfWeek'], axis=1 )

test_df.drop(['Month', 'DayofMonth',"DayOfWeek"],axis=1, inplace=True)

In [11]:
train_df.head()

,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,sin_hour,cos_hour,sin_min,cos_min,DayMonth,MonthWeek
0,AA,ATL,DFW,732,N,-0.965926,2.588190e-01,-0.406737,-9.135455e-01,c-8 c-21,c-8 c-7
1,US,PIT,MCO,834,N,-0.707107,-7.071068e-01,-0.951057,3.090170e-01,c-4 c-20,c-4 c-3
2,XE,RDU,CLE,416,N,-0.500000,-8.660254e-01,0.743145,-6.691306e-01,c-9 c-2,c-9 c-5
3,OO,DEN,MEM,872,N,0.500000,-8.660254e-01,1.000000,2.832769e-16,c-11 c-25,c-11 c-6
4,WN,MDW,OMA,423,Y,-1.000000,-1.836970e-16,0.207912,-9.781476e-01,c-10 c-7,c-10 c-6


In [12]:
test_df.head()

,UniqueCarrier,Origin,Dest,Distance,sin_hour,cos_hour,sin_min,cos_min,DayMonth,MonthWeek
0,YV,MRY,PHX,598,1.000000,6.123234e-17,1.000000,2.832769e-16,c-7 c-25,c-7 c-3
1,WN,LAS,HOU,1235,0.965926,-2.588190e-01,-0.809017,-5.877853e-01,c-4 c-17,c-4 c-2
2,MQ,GSP,ORD,577,1.000000,6.123234e-17,-0.809017,5.877853e-01,c-12 c-2,c-12 c-7
3,WN,BWI,MHT,377,-0.866025,-5.000000e-01,0.994522,1.045285e-01,c-3 c-25,c-3 c-7
4,UA,ORD,STL,258,-0.707107,-7.071068e-01,0.500000,8.660254e-01,c-6 c-6,c-6 c-3


In [13]:
cat_features = ["DayMonth", "MonthWeek", "UniqueCarrier", "Origin", "Dest"]

In [11]:
cat_features = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [21]:
df = pd.concat([train_df,test_df],axis=0)
le = LabelEncoder()
for col in cat_features:
    df[col] = le.fit_transform(df[col].astype(str))   

In [22]:
train_index = train_df.shape[0]

In [23]:
train_df = df.iloc[:train_index,:]
test_df = df.iloc[train_index:,:].drop('dep_delayed_15min',axis=1)

In [24]:
train_df.dep_delayed_15min.value_counts()

N    80956
Y    19044
Name: dep_delayed_15min, dtype: int64

In [62]:
train_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,sin_hour,cos_hour,sin_min,cos_min,DayMonth,MonthWeek
0,c-8,c-21,c-7,AA,ATL,DFW,732,N,-0.965926,2.588190e-01,-0.406737,-9.135455e-01,c-8 c-21,c-8 c-7
1,c-4,c-20,c-3,US,PIT,MCO,834,N,-0.707107,-7.071068e-01,-0.951057,3.090170e-01,c-4 c-20,c-4 c-3
2,c-9,c-2,c-5,XE,RDU,CLE,416,N,-0.500000,-8.660254e-01,0.743145,-6.691306e-01,c-9 c-2,c-9 c-5
3,c-11,c-25,c-6,OO,DEN,MEM,872,N,0.500000,-8.660254e-01,1.000000,2.832769e-16,c-11 c-25,c-11 c-6
4,c-10,c-7,c-6,WN,MDW,OMA,423,Y,-1.000000,-1.836970e-16,0.207912,-9.781476e-01,c-10 c-7,c-10 c-6


In [26]:
test_df.head()

,DayMonth,Dest,Distance,MonthWeek,Origin,UniqueCarrier,cos_hour,cos_min,sin_hour,sin_min
0,290,222,598,65,198,22,6.123234e-17,2.832769e-16,1.000000,1.000000
1,190,132,1235,43,159,20,-2.588190e-01,-5.877853e-01,0.965926,-0.809017
2,103,213,577,27,122,13,6.123234e-17,5.877853e-01,1.000000,-0.809017
3,168,188,377,41,49,20,-5.000000e-01,1.045285e-01,-0.866025,0.994522
4,269,276,258,58,213,18,-7.071068e-01,8.660254e-01,-0.707107,0.500000


In [14]:
X_train = train_df.drop('dep_delayed_15min',axis=1)
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0})

In [15]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [16]:
categorical_features_pos = [X_train.columns.get_loc(i) for i in cat_features] 

## LightGBM

#### Baselilne

In [30]:
%%time
lgb_clf = LGBMClassifier(random_state=17, categorical_feature=categorical_features_pos)
lgb_clf.fit(X_train_part, y_train_part)

Wall time: 911 ms


In [31]:
roc_auc_score(y_valid,lgb_clf.predict_proba(X_valid)[:,1])

0.732004796247437

#### Grid Search

In [ ]:
gridParams = {
    'num_leaves':[7,15,31],
    'max_depth':[3,4,5,-1],
    'reg_alpha' : [0.6,0.7,0.8],
    'reg_lambda':[0.6,0.7,0.8]
    }

In [ ]:
mdl = LGBMClassifier(boosting_type= 'gbdt', objective = 'binary', random_state=17,
                    categorical_feature=categorical_features_pos,n_jobs=-1)

grid_searcher = GridSearchCV(estimator=mdl, param_grid=gridParams, cv=5, verbose=1, n_jobs=-1,scoring='roc_auc')

grid_searcher.fit(X_train_part, y_train_part)

In [ ]:
grid_searcher.best_params_, grid_searcher.best_score_

In [ ]:
roc_auc_score(y_valid, grid_searcher.predict_proba(X_valid)[:,1])

#### Choosing learning rate

In [23]:
num_iterations = 500
lgb_clf2 = LGBMClassifier(boosting_type= 'gbdt', objective = 'binary',
                          random_state=17,num_leaves=15, max_depth=4,reg_lambda=0.8,
                          reg_alpha=0.8,n_estimators=num_iterations,
                          n_jobs=-1,categorical_feature=categorical_features_pos)

param_grid2 = {'learning_rate': np.logspace(-3, 0, 5)}
grid_searcher2 = GridSearchCV(estimator=lgb_clf2, param_grid=param_grid2, scoring='roc_auc',
                               cv=5, verbose=1, n_jobs=-1)

grid_searcher2.fit(X_train_part, y_train_part)

print(grid_searcher2.best_params_, grid_searcher2.best_score_)
print(roc_auc_score(y_valid, grid_searcher2.predict_proba(X_valid)[:,1]))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.3min finished


{'learning_rate': 0.03162277660168379} 0.731487481913914
0.7248353166877965


#### Final training on full data

In [50]:
final_lgb  = LGBMClassifier(boosting_type= 'gbdt', objective = 'binary',
                          random_state=17, reg_lambda=0.8,num_leaves=15, max_depth=4,
                          n_estimators=500, reg_alpha=0.8, learning_rate = 0.03162277660168379,
                          n_jobs=-1)

In [51]:
final_lgb.fit(X_train,y_train)
lightboost_test_pred = final_lgb.predict_proba(test_df)[:, 1]

## Catboost

#### Baseline

In [29]:
cb_clf = CatBoostClassifier(verbose=True,bagging_temperature=0.2,depth=9, iterations=2500)
cb_clf.fit(X_train_part, y_train_part,cat_features=categorical_features_pos)

0:	learn: 0.6785515	total: 245ms	remaining: 10m 12s
1:	learn: 0.6648022	total: 429ms	remaining: 8m 55s
2:	learn: 0.6519115	total: 637ms	remaining: 8m 50s
3:	learn: 0.6396781	total: 876ms	remaining: 9m 6s
4:	learn: 0.6281420	total: 1.15s	remaining: 9m 32s
5:	learn: 0.6173606	total: 1.41s	remaining: 9m 47s
6:	learn: 0.6070888	total: 1.65s	remaining: 9m 48s
7:	learn: 0.5977430	total: 1.85s	remaining: 9m 35s
8:	learn: 0.5885671	total: 2.07s	remaining: 9m 32s
9:	learn: 0.5800148	total: 2.3s	remaining: 9m 33s
10:	learn: 0.5718730	total: 2.52s	remaining: 9m 29s
11:	learn: 0.5643162	total: 2.76s	remaining: 9m 31s
12:	learn: 0.5571791	total: 2.96s	remaining: 9m 26s
13:	learn: 0.5505017	total: 3.19s	remaining: 9m 27s
14:	learn: 0.5441259	total: 3.41s	remaining: 9m 24s
15:	learn: 0.5385754	total: 3.66s	remaining: 9m 28s
16:	learn: 0.5329387	total: 3.88s	remaining: 9m 27s
17:	learn: 0.5278542	total: 4.13s	remaining: 9m 29s
18:	learn: 0.5231756	total: 4.33s	remaining: 9m 26s
19:	learn: 0.5184367	to

158:	learn: 0.4101602	total: 40.4s	remaining: 9m 54s
159:	learn: 0.4100661	total: 40.8s	remaining: 9m 56s
160:	learn: 0.4098915	total: 41.1s	remaining: 9m 57s
161:	learn: 0.4097401	total: 41.5s	remaining: 9m 58s
162:	learn: 0.4096121	total: 42s	remaining: 10m 2s
163:	learn: 0.4094526	total: 42.4s	remaining: 10m 3s
164:	learn: 0.4093634	total: 42.7s	remaining: 10m 4s
165:	learn: 0.4092427	total: 43.1s	remaining: 10m 6s
166:	learn: 0.4090849	total: 43.5s	remaining: 10m 7s
167:	learn: 0.4089074	total: 43.8s	remaining: 10m 7s
168:	learn: 0.4088080	total: 44.1s	remaining: 10m 8s
169:	learn: 0.4086646	total: 44.5s	remaining: 10m 9s
170:	learn: 0.4085479	total: 44.8s	remaining: 10m 10s
171:	learn: 0.4084017	total: 45.1s	remaining: 10m 10s
172:	learn: 0.4082576	total: 45.5s	remaining: 10m 12s
173:	learn: 0.4080904	total: 45.9s	remaining: 10m 12s
174:	learn: 0.4079338	total: 46.2s	remaining: 10m 13s
175:	learn: 0.4077870	total: 46.5s	remaining: 10m 14s
176:	learn: 0.4075991	total: 46.8s	remaini

310:	learn: 0.3940392	total: 1m 30s	remaining: 10m 35s
311:	learn: 0.3939425	total: 1m 30s	remaining: 10m 35s
312:	learn: 0.3937976	total: 1m 31s	remaining: 10m 35s
313:	learn: 0.3937153	total: 1m 31s	remaining: 10m 36s
314:	learn: 0.3936417	total: 1m 31s	remaining: 10m 36s
315:	learn: 0.3935630	total: 1m 32s	remaining: 10m 36s
316:	learn: 0.3934873	total: 1m 32s	remaining: 10m 36s
317:	learn: 0.3934161	total: 1m 32s	remaining: 10m 37s
318:	learn: 0.3933472	total: 1m 33s	remaining: 10m 37s
319:	learn: 0.3932581	total: 1m 33s	remaining: 10m 37s
320:	learn: 0.3931713	total: 1m 33s	remaining: 10m 37s
321:	learn: 0.3930684	total: 1m 34s	remaining: 10m 37s
322:	learn: 0.3929884	total: 1m 34s	remaining: 10m 37s
323:	learn: 0.3929141	total: 1m 34s	remaining: 10m 37s
324:	learn: 0.3928083	total: 1m 35s	remaining: 10m 37s
325:	learn: 0.3926328	total: 1m 35s	remaining: 10m 37s
326:	learn: 0.3925626	total: 1m 35s	remaining: 10m 37s
327:	learn: 0.3924212	total: 1m 36s	remaining: 10m 37s
328:	learn

462:	learn: 0.3803865	total: 2m 11s	remaining: 9m 36s
463:	learn: 0.3803067	total: 2m 11s	remaining: 9m 36s
464:	learn: 0.3802515	total: 2m 11s	remaining: 9m 35s
465:	learn: 0.3802071	total: 2m 11s	remaining: 9m 35s
466:	learn: 0.3800882	total: 2m 12s	remaining: 9m 35s
467:	learn: 0.3800380	total: 2m 12s	remaining: 9m 35s
468:	learn: 0.3799844	total: 2m 12s	remaining: 9m 35s
469:	learn: 0.3798995	total: 2m 12s	remaining: 9m 34s
470:	learn: 0.3798252	total: 2m 13s	remaining: 9m 34s
471:	learn: 0.3797907	total: 2m 13s	remaining: 9m 34s
472:	learn: 0.3797035	total: 2m 13s	remaining: 9m 33s
473:	learn: 0.3796186	total: 2m 14s	remaining: 9m 33s
474:	learn: 0.3795687	total: 2m 14s	remaining: 9m 32s
475:	learn: 0.3795109	total: 2m 14s	remaining: 9m 32s
476:	learn: 0.3794412	total: 2m 14s	remaining: 9m 31s
477:	learn: 0.3792984	total: 2m 15s	remaining: 9m 31s
478:	learn: 0.3792232	total: 2m 15s	remaining: 9m 31s
479:	learn: 0.3791442	total: 2m 15s	remaining: 9m 30s
480:	learn: 0.3790393	total:

615:	learn: 0.3684935	total: 2m 50s	remaining: 8m 40s
616:	learn: 0.3684224	total: 2m 50s	remaining: 8m 39s
617:	learn: 0.3683517	total: 2m 50s	remaining: 8m 39s
618:	learn: 0.3682406	total: 2m 50s	remaining: 8m 38s
619:	learn: 0.3681942	total: 2m 51s	remaining: 8m 38s
620:	learn: 0.3681171	total: 2m 51s	remaining: 8m 38s
621:	learn: 0.3680624	total: 2m 51s	remaining: 8m 37s
622:	learn: 0.3680116	total: 2m 51s	remaining: 8m 37s
623:	learn: 0.3679398	total: 2m 52s	remaining: 8m 37s
624:	learn: 0.3678800	total: 2m 52s	remaining: 8m 36s
625:	learn: 0.3678140	total: 2m 52s	remaining: 8m 36s
626:	learn: 0.3677692	total: 2m 52s	remaining: 8m 36s
627:	learn: 0.3676800	total: 2m 53s	remaining: 8m 36s
628:	learn: 0.3675830	total: 2m 53s	remaining: 8m 35s
629:	learn: 0.3675168	total: 2m 53s	remaining: 8m 35s
630:	learn: 0.3674725	total: 2m 53s	remaining: 8m 35s
631:	learn: 0.3674188	total: 2m 54s	remaining: 8m 34s
632:	learn: 0.3673462	total: 2m 54s	remaining: 8m 34s
633:	learn: 0.3672966	total:

...

1829:	learn: 0.2853401	total: 8m 32s	remaining: 3m 7s
1830:	learn: 0.2852671	total: 8m 32s	remaining: 3m 7s
1831:	learn: 0.2852279	total: 8m 32s	remaining: 3m 7s
1832:	learn: 0.2851892	total: 8m 33s	remaining: 3m 6s
1833:	learn: 0.2850832	total: 8m 33s	remaining: 3m 6s
1834:	learn: 0.2850251	total: 8m 33s	remaining: 3m 6s
1835:	learn: 0.2849837	total: 8m 33s	remaining: 3m 5s
1836:	learn: 0.2849152	total: 8m 34s	remaining: 3m 5s
1837:	learn: 0.2848813	total: 8m 34s	remaining: 3m 5s
1838:	learn: 0.2848418	total: 8m 34s	remaining: 3m 4s
1839:	learn: 0.2847677	total: 8m 34s	remaining: 3m 4s
1840:	learn: 0.2846514	total: 8m 35s	remaining: 3m 4s
1841:	learn: 0.2845763	total: 8m 35s	remaining: 3m 4s
1842:	learn: 0.2845442	total: 8m 35s	remaining: 3m 3s
1843:	learn: 0.2844709	total: 8m 36s	remaining: 3m 3s
1844:	learn: 0.2844239	total: 8m 36s	remaining: 3m 3s
1845:	learn: 0.2843766	total: 8m 37s	remaining: 3m 3s
1846:	learn: 0.2842918	total: 8m 37s	remaining: 3m 2s
1847:	learn: 0.2842259	total

1980:	learn: 0.2767794	total: 9m 15s	remaining: 2m 25s
1981:	learn: 0.2767262	total: 9m 16s	remaining: 2m 25s
1982:	learn: 0.2766863	total: 9m 16s	remaining: 2m 25s
1983:	learn: 0.2766437	total: 9m 16s	remaining: 2m 24s
1984:	learn: 0.2766163	total: 9m 16s	remaining: 2m 24s
1985:	learn: 0.2765643	total: 9m 17s	remaining: 2m 24s
1986:	learn: 0.2765322	total: 9m 17s	remaining: 2m 23s
1987:	learn: 0.2764603	total: 9m 17s	remaining: 2m 23s
1988:	learn: 0.2763820	total: 9m 17s	remaining: 2m 23s
1989:	learn: 0.2763333	total: 9m 18s	remaining: 2m 23s
1990:	learn: 0.2762769	total: 9m 18s	remaining: 2m 22s
1991:	learn: 0.2762045	total: 9m 18s	remaining: 2m 22s
1992:	learn: 0.2761446	total: 9m 18s	remaining: 2m 22s
1993:	learn: 0.2760642	total: 9m 19s	remaining: 2m 21s
1994:	learn: 0.2760188	total: 9m 19s	remaining: 2m 21s
1995:	learn: 0.2759549	total: 9m 19s	remaining: 2m 21s
1996:	learn: 0.2759135	total: 9m 19s	remaining: 2m 21s
1997:	learn: 0.2758513	total: 9m 20s	remaining: 2m 20s
1998:	lear

2130:	learn: 0.2684407	total: 9m 54s	remaining: 1m 43s
2131:	learn: 0.2684015	total: 9m 55s	remaining: 1m 42s
2132:	learn: 0.2683277	total: 9m 55s	remaining: 1m 42s
2133:	learn: 0.2682649	total: 9m 55s	remaining: 1m 42s
2134:	learn: 0.2681978	total: 9m 55s	remaining: 1m 41s
2135:	learn: 0.2681364	total: 9m 56s	remaining: 1m 41s
2136:	learn: 0.2680526	total: 9m 56s	remaining: 1m 41s
2137:	learn: 0.2680280	total: 9m 56s	remaining: 1m 41s
2138:	learn: 0.2679914	total: 9m 56s	remaining: 1m 40s
2139:	learn: 0.2679110	total: 9m 57s	remaining: 1m 40s
2140:	learn: 0.2678525	total: 9m 57s	remaining: 1m 40s
2141:	learn: 0.2678256	total: 9m 57s	remaining: 1m 39s
2142:	learn: 0.2678032	total: 9m 58s	remaining: 1m 39s
2143:	learn: 0.2677626	total: 9m 58s	remaining: 1m 39s
2144:	learn: 0.2676962	total: 9m 58s	remaining: 1m 39s
2145:	learn: 0.2676452	total: 9m 58s	remaining: 1m 38s
2146:	learn: 0.2676097	total: 9m 59s	remaining: 1m 38s
2147:	learn: 0.2675511	total: 9m 59s	remaining: 1m 38s
2148:	lear

2279:	learn: 0.2604720	total: 10m 34s	remaining: 1m 1s
2280:	learn: 0.2604654	total: 10m 34s	remaining: 1m
2281:	learn: 0.2604212	total: 10m 35s	remaining: 1m
2282:	learn: 0.2603634	total: 10m 35s	remaining: 1m
2283:	learn: 0.2603141	total: 10m 35s	remaining: 1m
2284:	learn: 0.2602715	total: 10m 36s	remaining: 59.8s
2285:	learn: 0.2601984	total: 10m 36s	remaining: 59.6s
2286:	learn: 0.2601664	total: 10m 36s	remaining: 59.3s
2287:	learn: 0.2601049	total: 10m 36s	remaining: 59s
2288:	learn: 0.2600366	total: 10m 37s	remaining: 58.7s
2289:	learn: 0.2599660	total: 10m 37s	remaining: 58.4s
2290:	learn: 0.2598705	total: 10m 37s	remaining: 58.2s
2291:	learn: 0.2598178	total: 10m 37s	remaining: 57.9s
2292:	learn: 0.2597380	total: 10m 38s	remaining: 57.6s
2293:	learn: 0.2596671	total: 10m 38s	remaining: 57.3s
2294:	learn: 0.2596172	total: 10m 38s	remaining: 57s
2295:	learn: 0.2595489	total: 10m 38s	remaining: 56.8s
2296:	learn: 0.2594856	total: 10m 39s	remaining: 56.5s
2297:	learn: 0.2594313	tot

2430:	learn: 0.2523504	total: 11m 15s	remaining: 19.2s
2431:	learn: 0.2523184	total: 11m 15s	remaining: 18.9s
2432:	learn: 0.2522229	total: 11m 15s	remaining: 18.6s
2433:	learn: 0.2521721	total: 11m 15s	remaining: 18.3s
2434:	learn: 0.2520871	total: 11m 16s	remaining: 18.1s
2435:	learn: 0.2520526	total: 11m 16s	remaining: 17.8s
2436:	learn: 0.2519820	total: 11m 16s	remaining: 17.5s
2437:	learn: 0.2519399	total: 11m 17s	remaining: 17.2s
2438:	learn: 0.2519178	total: 11m 17s	remaining: 16.9s
2439:	learn: 0.2518774	total: 11m 17s	remaining: 16.7s
2440:	learn: 0.2518203	total: 11m 17s	remaining: 16.4s
2441:	learn: 0.2518097	total: 11m 18s	remaining: 16.1s
2442:	learn: 0.2517303	total: 11m 18s	remaining: 15.8s
2443:	learn: 0.2516543	total: 11m 18s	remaining: 15.5s
2444:	learn: 0.2516080	total: 11m 18s	remaining: 15.3s
2445:	learn: 0.2515435	total: 11m 19s	remaining: 15s
2446:	learn: 0.2514760	total: 11m 19s	remaining: 14.7s
2447:	learn: 0.2514267	total: 11m 19s	remaining: 14.4s
2448:	learn:

In [30]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])
# with sine and cosine fetures without ss + baggine temperature0.2+depth8 +newfeature #0.7652695613286953

0.7660989523441822

In [39]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])
# with sine and cosine fetures without ss + baggine temperature0.2+depth8 +newfeature 2 #0.764167062757251

0.7641670627572512

In [41]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])# with sine and cosine fetures without ss + baggine temperature0.1+depth9 #0.7630835112491842

0.763689033861166

In [30]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])# with sine and cosine fetures without ss + baggine temperature+depth9 #0.7630835112491842

0.7630835112491842

In [58]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])# with sine and cosine fetures without ss + baggine temperature #0.7581006294349376

0.7581006294349376

In [46]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])# with sine and cosine fetures without ss #0.7548935415503406

0.7548935415503406

In [24]:
roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])# with raw min and hour #0.7538128262986981

0.7538128262986981

In [27]:
# roc_auc_score(y_valid,cb_clf.predict_proba(X_valid)[:,1])#with sine and cosine features 0.7548935415503406

0.7548935415503406

#### Grid Search

In [59]:
params = {'depth':[5,6,7],
          'learning_rate': np.logspace(-3,0,5)}

In [60]:
cb_clf2 = CatBoostClassifier(random_state=17, cat_features=categorical_features_pos,thread_count=4,verbose=True)

grid_searcher_cb = GridSearchCV(estimator=cb_clf2, param_grid=params, cv=5, verbose=1, n_jobs=-1,scoring='roc_auc')

grid_searcher_cb.fit(X_train_part, y_train_part)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 72.2min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 137.2min finished


0:	learn: 0.6748563	total: 217ms	remaining: 3m 36s
1:	learn: 0.6579328	total: 407ms	remaining: 3m 23s
2:	learn: 0.6419803	total: 576ms	remaining: 3m 11s
3:	learn: 0.6275371	total: 763ms	remaining: 3m 9s
4:	learn: 0.6143827	total: 914ms	remaining: 3m 1s
5:	learn: 0.6019606	total: 1.13s	remaining: 3m 8s
6:	learn: 0.5904348	total: 1.32s	remaining: 3m 7s
7:	learn: 0.5797894	total: 1.56s	remaining: 3m 13s
8:	learn: 0.5697930	total: 1.74s	remaining: 3m 11s
9:	learn: 0.5613264	total: 1.93s	remaining: 3m 11s
10:	learn: 0.5528969	total: 2.12s	remaining: 3m 10s
11:	learn: 0.5456600	total: 2.29s	remaining: 3m 8s
12:	learn: 0.5383206	total: 2.44s	remaining: 3m 5s
13:	learn: 0.5313770	total: 2.62s	remaining: 3m 4s
14:	learn: 0.5252897	total: 2.8s	remaining: 3m 4s
15:	learn: 0.5197050	total: 3s	remaining: 3m 4s
16:	learn: 0.5146819	total: 3.15s	remaining: 3m 2s
17:	learn: 0.5096692	total: 3.34s	remaining: 3m 2s
18:	learn: 0.5052579	total: 3.55s	remaining: 3m 3s
19:	learn: 0.5011001	total: 3.74s	rema

159:	learn: 0.4206634	total: 27.6s	remaining: 2m 25s
160:	learn: 0.4206020	total: 27.8s	remaining: 2m 24s
161:	learn: 0.4205588	total: 27.9s	remaining: 2m 24s
162:	learn: 0.4204377	total: 28.1s	remaining: 2m 24s
163:	learn: 0.4203716	total: 28.3s	remaining: 2m 24s
164:	learn: 0.4203001	total: 28.5s	remaining: 2m 24s
165:	learn: 0.4202512	total: 28.6s	remaining: 2m 23s
166:	learn: 0.4201773	total: 28.8s	remaining: 2m 23s
167:	learn: 0.4200914	total: 29s	remaining: 2m 23s
168:	learn: 0.4200437	total: 29.1s	remaining: 2m 23s
169:	learn: 0.4199772	total: 29.3s	remaining: 2m 23s
170:	learn: 0.4199214	total: 29.5s	remaining: 2m 22s
171:	learn: 0.4198250	total: 29.6s	remaining: 2m 22s
172:	learn: 0.4197428	total: 29.8s	remaining: 2m 22s
173:	learn: 0.4196372	total: 30s	remaining: 2m 22s
174:	learn: 0.4195578	total: 30.2s	remaining: 2m 22s
175:	learn: 0.4194753	total: 30.3s	remaining: 2m 22s
176:	learn: 0.4194005	total: 30.5s	remaining: 2m 21s
177:	learn: 0.4193587	total: 30.7s	remaining: 2m 2

316:	learn: 0.4112459	total: 54.3s	remaining: 1m 57s
317:	learn: 0.4112261	total: 54.5s	remaining: 1m 56s
318:	learn: 0.4111942	total: 54.6s	remaining: 1m 56s
319:	learn: 0.4111355	total: 54.8s	remaining: 1m 56s
320:	learn: 0.4110794	total: 55s	remaining: 1m 56s
321:	learn: 0.4109980	total: 55.2s	remaining: 1m 56s
322:	learn: 0.4109723	total: 55.4s	remaining: 1m 56s
323:	learn: 0.4109343	total: 55.5s	remaining: 1m 55s
324:	learn: 0.4109109	total: 55.6s	remaining: 1m 55s
325:	learn: 0.4108552	total: 55.8s	remaining: 1m 55s
326:	learn: 0.4108069	total: 55.9s	remaining: 1m 55s
327:	learn: 0.4107538	total: 56.1s	remaining: 1m 54s
328:	learn: 0.4107256	total: 56.3s	remaining: 1m 54s
329:	learn: 0.4106781	total: 56.4s	remaining: 1m 54s
330:	learn: 0.4106501	total: 56.6s	remaining: 1m 54s
331:	learn: 0.4106227	total: 56.8s	remaining: 1m 54s
332:	learn: 0.4105520	total: 56.9s	remaining: 1m 54s
333:	learn: 0.4105179	total: 57.1s	remaining: 1m 53s
334:	learn: 0.4104527	total: 57.3s	remaining: 1m

470:	learn: 0.4039245	total: 1m 21s	remaining: 1m 31s
471:	learn: 0.4038853	total: 1m 21s	remaining: 1m 31s
472:	learn: 0.4038542	total: 1m 21s	remaining: 1m 30s
473:	learn: 0.4038265	total: 1m 21s	remaining: 1m 30s
474:	learn: 0.4037740	total: 1m 21s	remaining: 1m 30s
475:	learn: 0.4037432	total: 1m 22s	remaining: 1m 30s
476:	learn: 0.4037045	total: 1m 22s	remaining: 1m 30s
477:	learn: 0.4036368	total: 1m 22s	remaining: 1m 30s
478:	learn: 0.4035931	total: 1m 22s	remaining: 1m 29s
479:	learn: 0.4035645	total: 1m 22s	remaining: 1m 29s
480:	learn: 0.4034864	total: 1m 22s	remaining: 1m 29s
481:	learn: 0.4034513	total: 1m 23s	remaining: 1m 29s
482:	learn: 0.4034046	total: 1m 23s	remaining: 1m 29s
483:	learn: 0.4033306	total: 1m 23s	remaining: 1m 29s
484:	learn: 0.4032848	total: 1m 23s	remaining: 1m 28s
485:	learn: 0.4032428	total: 1m 23s	remaining: 1m 28s
486:	learn: 0.4031913	total: 1m 24s	remaining: 1m 28s
487:	learn: 0.4030984	total: 1m 24s	remaining: 1m 28s
488:	learn: 0.4030123	total:

624:	learn: 0.3977890	total: 1m 47s	remaining: 1m 4s
625:	learn: 0.3977558	total: 1m 47s	remaining: 1m 4s
626:	learn: 0.3977274	total: 1m 48s	remaining: 1m 4s
627:	learn: 0.3976753	total: 1m 48s	remaining: 1m 4s
628:	learn: 0.3976197	total: 1m 48s	remaining: 1m 3s
629:	learn: 0.3975866	total: 1m 48s	remaining: 1m 3s
630:	learn: 0.3975502	total: 1m 48s	remaining: 1m 3s
631:	learn: 0.3975293	total: 1m 48s	remaining: 1m 3s
632:	learn: 0.3974728	total: 1m 49s	remaining: 1m 3s
633:	learn: 0.3974550	total: 1m 49s	remaining: 1m 3s
634:	learn: 0.3974309	total: 1m 49s	remaining: 1m 2s
635:	learn: 0.3973904	total: 1m 49s	remaining: 1m 2s
636:	learn: 0.3973581	total: 1m 49s	remaining: 1m 2s
637:	learn: 0.3973278	total: 1m 49s	remaining: 1m 2s
638:	learn: 0.3973036	total: 1m 50s	remaining: 1m 2s
639:	learn: 0.3972405	total: 1m 50s	remaining: 1m 2s
640:	learn: 0.3972031	total: 1m 50s	remaining: 1m 1s
641:	learn: 0.3971719	total: 1m 50s	remaining: 1m 1s
642:	learn: 0.3971458	total: 1m 50s	remaining:

781:	learn: 0.3916359	total: 2m 15s	remaining: 37.7s
782:	learn: 0.3916071	total: 2m 15s	remaining: 37.5s
783:	learn: 0.3915863	total: 2m 15s	remaining: 37.3s
784:	learn: 0.3915482	total: 2m 15s	remaining: 37.1s
785:	learn: 0.3915073	total: 2m 15s	remaining: 37s
786:	learn: 0.3914428	total: 2m 15s	remaining: 36.8s
787:	learn: 0.3913940	total: 2m 16s	remaining: 36.6s
788:	learn: 0.3913466	total: 2m 16s	remaining: 36.5s
789:	learn: 0.3913156	total: 2m 16s	remaining: 36.3s
790:	learn: 0.3912895	total: 2m 16s	remaining: 36.1s
791:	learn: 0.3912413	total: 2m 16s	remaining: 35.9s
792:	learn: 0.3911894	total: 2m 17s	remaining: 35.8s
793:	learn: 0.3911424	total: 2m 17s	remaining: 35.6s
794:	learn: 0.3911134	total: 2m 17s	remaining: 35.4s
795:	learn: 0.3910585	total: 2m 17s	remaining: 35.3s
796:	learn: 0.3910131	total: 2m 17s	remaining: 35.1s
797:	learn: 0.3909730	total: 2m 17s	remaining: 34.9s
798:	learn: 0.3909611	total: 2m 18s	remaining: 34.7s
799:	learn: 0.3909407	total: 2m 18s	remaining: 3

937:	learn: 0.3856940	total: 2m 43s	remaining: 10.8s
938:	learn: 0.3856660	total: 2m 43s	remaining: 10.6s
939:	learn: 0.3856382	total: 2m 43s	remaining: 10.4s
940:	learn: 0.3855986	total: 2m 43s	remaining: 10.3s
941:	learn: 0.3855692	total: 2m 44s	remaining: 10.1s
942:	learn: 0.3855293	total: 2m 44s	remaining: 9.92s
943:	learn: 0.3854749	total: 2m 44s	remaining: 9.75s
944:	learn: 0.3854539	total: 2m 44s	remaining: 9.58s
945:	learn: 0.3854035	total: 2m 44s	remaining: 9.4s
946:	learn: 0.3853561	total: 2m 44s	remaining: 9.23s
947:	learn: 0.3853090	total: 2m 45s	remaining: 9.06s
948:	learn: 0.3852781	total: 2m 45s	remaining: 8.88s
949:	learn: 0.3852509	total: 2m 45s	remaining: 8.71s
950:	learn: 0.3852044	total: 2m 45s	remaining: 8.53s
951:	learn: 0.3851876	total: 2m 45s	remaining: 8.36s
952:	learn: 0.3851510	total: 2m 46s	remaining: 8.19s
953:	learn: 0.3850939	total: 2m 46s	remaining: 8.02s
954:	learn: 0.3850481	total: 2m 46s	remaining: 7.84s
955:	learn: 0.3850097	total: 2m 46s	remaining: 

GridSearchCV(cv=5, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x000001E12F47E7F0>,
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'depth': [5, 6, 7], 'learning_rate': array([0.001  , 0.00562, 0.03162, 0.17783, 1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [61]:
grid_searcher_cb.best_params_, grid_searcher_cb.best_score_

({'depth': 7, 'learning_rate': 0.03162277660168379}, 0.755767815517814)

#### Full Data Training 

In [31]:
cb_clf.fit(X_train,y_train,cat_features=categorical_features_pos)
catboost_test_pred = cb_clf.predict_proba(test_df)[:, 1]

0:	learn: 0.6770281	total: 285ms	remaining: 11m 53s
1:	learn: 0.6620144	total: 603ms	remaining: 12m 32s
2:	learn: 0.6480321	total: 896ms	remaining: 12m 25s
3:	learn: 0.6348907	total: 1.19s	remaining: 12m 21s
4:	learn: 0.6226784	total: 1.51s	remaining: 12m 32s
5:	learn: 0.6111647	total: 1.76s	remaining: 12m 12s
6:	learn: 0.6006464	total: 2.02s	remaining: 11m 58s
7:	learn: 0.5906412	total: 2.31s	remaining: 12m
8:	learn: 0.5811079	total: 2.6s	remaining: 12m
9:	learn: 0.5721124	total: 2.9s	remaining: 12m 3s
10:	learn: 0.5638047	total: 3.21s	remaining: 12m 6s
11:	learn: 0.5561205	total: 3.43s	remaining: 11m 51s
12:	learn: 0.5487968	total: 3.73s	remaining: 11m 54s
13:	learn: 0.5419711	total: 4.05s	remaining: 11m 58s
14:	learn: 0.5355272	total: 4.4s	remaining: 12m 9s
15:	learn: 0.5294647	total: 4.7s	remaining: 12m 9s
16:	learn: 0.5235276	total: 5s	remaining: 12m 11s
17:	learn: 0.5181786	total: 5.25s	remaining: 12m 4s
18:	learn: 0.5130375	total: 5.54s	remaining: 12m 3s
19:	learn: 0.5080413	tot

156:	learn: 0.4099323	total: 48.5s	remaining: 12m 4s
157:	learn: 0.4097766	total: 48.9s	remaining: 12m 4s
158:	learn: 0.4096697	total: 49.1s	remaining: 12m 2s
159:	learn: 0.4094144	total: 49.4s	remaining: 12m 2s
160:	learn: 0.4093131	total: 49.7s	remaining: 12m 2s
161:	learn: 0.4091959	total: 50.1s	remaining: 12m 2s
162:	learn: 0.4091100	total: 50.4s	remaining: 12m 2s
163:	learn: 0.4090282	total: 50.7s	remaining: 12m 1s
164:	learn: 0.4089459	total: 50.9s	remaining: 12m
165:	learn: 0.4087946	total: 51.3s	remaining: 12m
166:	learn: 0.4085892	total: 51.6s	remaining: 12m
167:	learn: 0.4084465	total: 51.9s	remaining: 12m
168:	learn: 0.4083357	total: 52.2s	remaining: 11m 59s
169:	learn: 0.4082035	total: 52.5s	remaining: 11m 59s
170:	learn: 0.4080836	total: 52.7s	remaining: 11m 58s
171:	learn: 0.4079505	total: 53s	remaining: 11m 57s
172:	learn: 0.4077960	total: 53.4s	remaining: 11m 57s
173:	learn: 0.4076948	total: 53.7s	remaining: 11m 57s
174:	learn: 0.4075413	total: 54s	remaining: 11m 57s
17

308:	learn: 0.3935692	total: 1m 38s	remaining: 11m 36s
309:	learn: 0.3934900	total: 1m 38s	remaining: 11m 36s
310:	learn: 0.3933037	total: 1m 39s	remaining: 11m 37s
311:	learn: 0.3931920	total: 1m 39s	remaining: 11m 37s
312:	learn: 0.3931348	total: 1m 39s	remaining: 11m 36s
313:	learn: 0.3930438	total: 1m 40s	remaining: 11m 36s
314:	learn: 0.3929591	total: 1m 40s	remaining: 11m 35s
315:	learn: 0.3928286	total: 1m 40s	remaining: 11m 35s
316:	learn: 0.3927610	total: 1m 40s	remaining: 11m 35s
317:	learn: 0.3926791	total: 1m 41s	remaining: 11m 34s
318:	learn: 0.3925408	total: 1m 41s	remaining: 11m 34s
319:	learn: 0.3924267	total: 1m 41s	remaining: 11m 34s
320:	learn: 0.3923764	total: 1m 42s	remaining: 11m 33s
321:	learn: 0.3923179	total: 1m 42s	remaining: 11m 33s
322:	learn: 0.3922663	total: 1m 42s	remaining: 11m 33s
323:	learn: 0.3921763	total: 1m 43s	remaining: 11m 33s
324:	learn: 0.3921176	total: 1m 43s	remaining: 11m 32s
325:	learn: 0.3920692	total: 1m 43s	remaining: 11m 32s
326:	learn

458:	learn: 0.3815537	total: 2m 36s	remaining: 11m 34s
459:	learn: 0.3814881	total: 2m 36s	remaining: 11m 34s
460:	learn: 0.3814383	total: 2m 36s	remaining: 11m 33s
461:	learn: 0.3813722	total: 2m 37s	remaining: 11m 33s
462:	learn: 0.3813393	total: 2m 37s	remaining: 11m 33s
463:	learn: 0.3812464	total: 2m 38s	remaining: 11m 33s
464:	learn: 0.3811971	total: 2m 38s	remaining: 11m 33s
465:	learn: 0.3811388	total: 2m 38s	remaining: 11m 33s
466:	learn: 0.3810415	total: 2m 39s	remaining: 11m 33s
467:	learn: 0.3809608	total: 2m 39s	remaining: 11m 33s
468:	learn: 0.3808850	total: 2m 40s	remaining: 11m 32s
469:	learn: 0.3808088	total: 2m 40s	remaining: 11m 33s
470:	learn: 0.3807406	total: 2m 40s	remaining: 11m 33s
471:	learn: 0.3806602	total: 2m 41s	remaining: 11m 33s
472:	learn: 0.3806218	total: 2m 41s	remaining: 11m 33s
473:	learn: 0.3805660	total: 2m 42s	remaining: 11m 33s
474:	learn: 0.3804882	total: 2m 42s	remaining: 11m 33s
475:	learn: 0.3804341	total: 2m 43s	remaining: 11m 33s
476:	learn

609:	learn: 0.3710018	total: 3m 26s	remaining: 10m 40s
610:	learn: 0.3709084	total: 3m 27s	remaining: 10m 40s
611:	learn: 0.3708590	total: 3m 27s	remaining: 10m 40s
612:	learn: 0.3707789	total: 3m 27s	remaining: 10m 39s
613:	learn: 0.3707155	total: 3m 28s	remaining: 10m 39s
614:	learn: 0.3706397	total: 3m 28s	remaining: 10m 39s
615:	learn: 0.3705932	total: 3m 28s	remaining: 10m 38s
616:	learn: 0.3705634	total: 3m 29s	remaining: 10m 38s
617:	learn: 0.3704702	total: 3m 29s	remaining: 10m 38s
618:	learn: 0.3704097	total: 3m 29s	remaining: 10m 37s
619:	learn: 0.3703596	total: 3m 30s	remaining: 10m 37s
620:	learn: 0.3702637	total: 3m 30s	remaining: 10m 36s
621:	learn: 0.3701884	total: 3m 30s	remaining: 10m 36s
622:	learn: 0.3701307	total: 3m 31s	remaining: 10m 36s
623:	learn: 0.3700369	total: 3m 31s	remaining: 10m 35s
624:	learn: 0.3699850	total: 3m 31s	remaining: 10m 35s
625:	learn: 0.3698946	total: 3m 32s	remaining: 10m 34s
626:	learn: 0.3698185	total: 3m 32s	remaining: 10m 34s
627:	learn

...

1818:	learn: 0.2999917	total: 10m	remaining: 3m 44s
1819:	learn: 0.2999525	total: 10m	remaining: 3m 44s
1820:	learn: 0.2999311	total: 10m	remaining: 3m 44s
1821:	learn: 0.2998339	total: 10m 1s	remaining: 3m 43s
1822:	learn: 0.2997735	total: 10m 1s	remaining: 3m 43s
1823:	learn: 0.2997161	total: 10m 1s	remaining: 3m 43s
1824:	learn: 0.2996845	total: 10m 2s	remaining: 3m 42s
1825:	learn: 0.2996587	total: 10m 2s	remaining: 3m 42s
1826:	learn: 0.2996174	total: 10m 2s	remaining: 3m 42s
1827:	learn: 0.2995761	total: 10m 2s	remaining: 3m 41s
1828:	learn: 0.2995111	total: 10m 3s	remaining: 3m 41s
1829:	learn: 0.2994717	total: 10m 3s	remaining: 3m 41s
1830:	learn: 0.2994339	total: 10m 3s	remaining: 3m 40s
1831:	learn: 0.2993843	total: 10m 4s	remaining: 3m 40s
1832:	learn: 0.2993298	total: 10m 4s	remaining: 3m 40s
1833:	learn: 0.2992674	total: 10m 4s	remaining: 3m 39s
1834:	learn: 0.2991925	total: 10m 5s	remaining: 3m 39s
1835:	learn: 0.2991580	total: 10m 5s	remaining: 3m 39s
1836:	learn: 0.2991

1966:	learn: 0.2926755	total: 10m 48s	remaining: 2m 55s
1967:	learn: 0.2926302	total: 10m 48s	remaining: 2m 55s
1968:	learn: 0.2925846	total: 10m 49s	remaining: 2m 55s
1969:	learn: 0.2925224	total: 10m 49s	remaining: 2m 54s
1970:	learn: 0.2924495	total: 10m 49s	remaining: 2m 54s
1971:	learn: 0.2923818	total: 10m 50s	remaining: 2m 54s
1972:	learn: 0.2923418	total: 10m 50s	remaining: 2m 53s
1973:	learn: 0.2923174	total: 10m 50s	remaining: 2m 53s
1974:	learn: 0.2922667	total: 10m 51s	remaining: 2m 53s
1975:	learn: 0.2922072	total: 10m 51s	remaining: 2m 52s
1976:	learn: 0.2921834	total: 10m 51s	remaining: 2m 52s
1977:	learn: 0.2921668	total: 10m 51s	remaining: 2m 52s
1978:	learn: 0.2920997	total: 10m 52s	remaining: 2m 51s
1979:	learn: 0.2920345	total: 10m 52s	remaining: 2m 51s
1980:	learn: 0.2919570	total: 10m 52s	remaining: 2m 51s
1981:	learn: 0.2918952	total: 10m 53s	remaining: 2m 50s
1982:	learn: 0.2918479	total: 10m 53s	remaining: 2m 50s
1983:	learn: 0.2917950	total: 10m 54s	remaining:

2114:	learn: 0.2854304	total: 11m 43s	remaining: 2m 7s
2115:	learn: 0.2853686	total: 11m 43s	remaining: 2m 7s
2116:	learn: 0.2853096	total: 11m 44s	remaining: 2m 7s
2117:	learn: 0.2852383	total: 11m 44s	remaining: 2m 7s
2118:	learn: 0.2851999	total: 11m 44s	remaining: 2m 6s
2119:	learn: 0.2851523	total: 11m 45s	remaining: 2m 6s
2120:	learn: 0.2850911	total: 11m 45s	remaining: 2m 6s
2121:	learn: 0.2850541	total: 11m 46s	remaining: 2m 5s
2122:	learn: 0.2850140	total: 11m 46s	remaining: 2m 5s
2123:	learn: 0.2849662	total: 11m 47s	remaining: 2m 5s
2124:	learn: 0.2848885	total: 11m 47s	remaining: 2m 4s
2125:	learn: 0.2848509	total: 11m 47s	remaining: 2m 4s
2126:	learn: 0.2848014	total: 11m 48s	remaining: 2m 4s
2127:	learn: 0.2847121	total: 11m 48s	remaining: 2m 3s
2128:	learn: 0.2846679	total: 11m 49s	remaining: 2m 3s
2129:	learn: 0.2846475	total: 11m 49s	remaining: 2m 3s
2130:	learn: 0.2845908	total: 11m 50s	remaining: 2m 2s
2131:	learn: 0.2845645	total: 11m 50s	remaining: 2m 2s
2132:	lear

2262:	learn: 0.2787043	total: 12m 35s	remaining: 1m 19s
2263:	learn: 0.2786812	total: 12m 35s	remaining: 1m 18s
2264:	learn: 0.2786511	total: 12m 36s	remaining: 1m 18s
2265:	learn: 0.2786110	total: 12m 36s	remaining: 1m 18s
2266:	learn: 0.2785410	total: 12m 36s	remaining: 1m 17s
2267:	learn: 0.2785109	total: 12m 37s	remaining: 1m 17s
2268:	learn: 0.2784522	total: 12m 37s	remaining: 1m 17s
2269:	learn: 0.2784221	total: 12m 37s	remaining: 1m 16s
2270:	learn: 0.2783500	total: 12m 38s	remaining: 1m 16s
2271:	learn: 0.2782957	total: 12m 38s	remaining: 1m 16s
2272:	learn: 0.2782542	total: 12m 38s	remaining: 1m 15s
2273:	learn: 0.2782237	total: 12m 39s	remaining: 1m 15s
2274:	learn: 0.2781761	total: 12m 39s	remaining: 1m 15s
2275:	learn: 0.2781148	total: 12m 39s	remaining: 1m 14s
2276:	learn: 0.2780614	total: 12m 40s	remaining: 1m 14s
2277:	learn: 0.2780339	total: 12m 40s	remaining: 1m 14s
2278:	learn: 0.2779936	total: 12m 40s	remaining: 1m 13s
2279:	learn: 0.2779366	total: 12m 41s	remaining:

2413:	learn: 0.2718819	total: 13m 24s	remaining: 28.7s
2414:	learn: 0.2718248	total: 13m 24s	remaining: 28.3s
2415:	learn: 0.2717936	total: 13m 25s	remaining: 28s
2416:	learn: 0.2717546	total: 13m 25s	remaining: 27.7s
2417:	learn: 0.2717113	total: 13m 25s	remaining: 27.3s
2418:	learn: 0.2716709	total: 13m 26s	remaining: 27s
2419:	learn: 0.2716542	total: 13m 26s	remaining: 26.7s
2420:	learn: 0.2716220	total: 13m 26s	remaining: 26.3s
2421:	learn: 0.2715666	total: 13m 27s	remaining: 26s
2422:	learn: 0.2715224	total: 13m 27s	remaining: 25.7s
2423:	learn: 0.2714669	total: 13m 27s	remaining: 25.3s
2424:	learn: 0.2714195	total: 13m 28s	remaining: 25s
2425:	learn: 0.2713863	total: 13m 28s	remaining: 24.7s
2426:	learn: 0.2713247	total: 13m 28s	remaining: 24.3s
2427:	learn: 0.2712630	total: 13m 28s	remaining: 24s
2428:	learn: 0.2712161	total: 13m 29s	remaining: 23.7s
2429:	learn: 0.2711496	total: 13m 29s	remaining: 23.3s
2430:	learn: 0.2710859	total: 13m 29s	remaining: 23s
2431:	learn: 0.2710498

In [55]:
np.corrcoef(catboost_test_pred,lightboost_test_pred)

array([[1.        , 0.95341837],
       [0.95341837, 1.        ]])

In [32]:
pd.Series(catboost_test_pred, 
          name='dep_delayed_15min').to_csv('./assignment10/catboost_test_pred13.csv', 
                                           index_label='id', header=True)